In [ ]:
from statsmodels.graphics.mosaicplot import mosaic
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.rcParams['figure.figsize'] = [16, 5]
# plt.rcParams['figure.dpi'] = 140
plt.style.use('seaborn')
# file "Data Laporan Keuangan.xlsx"
file = 'Data Zonasi.xlsx'

In [ ]:
raw_df = pd.read_excel(file)
raw_df

## Tipe Data

In [ ]:
raw_df.info()

### Perbaikin Jenis Tidak Sesuai

In [ ]:
raw_df = pd.read_csv(file, parse_dates=["Tanggal Invoice"], thousands=",")
raw_df.info()

### Perbaiki nama kolom (hapus spasi / strip)

In [ ]:
raw_df.columns = [col_name.strip() for col_name in raw_df.columns]

In [ ]:
raw_df.info()

### Perbaiki Tipe Column 
- Discount ubah jadi numerical
- ID Salesman ubah jadi categorical

In [ ]:
replaceDash = raw_df.Discount.str.replace('-','0')
raw_df.Discount = replaceDash.astype('int64')
raw_df['ID Salesman'] = raw_df['ID Salesman'].astype('object')
raw_df['Jenis Kirim'] = raw_df['Jenis Kirim'].astype('object') 
raw_df.head()

In [ ]:
raw_df.info()

## Deskriptif Stats
### Numerical Variables

In [ ]:
raw_df.describe()

In [ ]:
desc_numerical = raw_df.describe().transpose()
desc_numerical

#### Ada yang kurang

In [ ]:
desc_numerical['range'] = desc_numerical['max'] - desc_numerical['min']
desc_numerical['iqr'] = desc_numerical['75%'] - desc_numerical['25%']

desc_numerical['skewness'] = [raw_df[col].skew() for col in desc_numerical.index]
desc_numerical['kurtosis'] = [raw_df[col].kurtosis() for col in desc_numerical.index]

# print
desc_numerical

### Categorical Variables

In [ ]:
desc_categorical = raw_df.describe(include="O").transpose()
desc_categorical

#### Apakah ada yang kurang?

## Visualizing Data
### Numerical

In [ ]:
raw_df[desc_numerical.index].hist()

plt.show()

In [ ]:
sns.ecdfplot(raw_df[["Ongkir/Unit", 'Discount', 'Quantity']])
plt.show()

#### terlalu skewed dan tinggi kurtosisnya untuk dianalisis, perlu transformasi nilai

In [ ]:
numerical_transform = raw_df[['Gross sales','Net sales','Total Sales']].apply(np.log1p)

In [ ]:
numerical_transform.plot(kind='kde')
plt.show()

In [ ]:
numerical_transform.plot(kind='box')
plt.show()

In [ ]:
sns.ecdfplot(numerical_transform)
plt.show()

In [ ]:
sns.scatterplot(x=raw_df['Total Sales'], y=raw_df['Gross sales'])

In [ ]:
sns.scatterplot(x=numerical_transform['Total Sales'], y=numerical_transform['Gross sales'])
plt.show()

In [ ]:
sns.scatterplot(x=numerical_transform['Total Sales'], y=numerical_transform['Gross sales'],
                hue=raw_df['Quantity'], palette='plasma')
plt.show()

### Outliers

In [ ]:
cols = numerical_transform.columns
for col in cols:
    col_zscore = col + '_zscore'
    numerical_transform[col_zscore] = (numerical_transform[col] - numerical_transform[col].mean())/numerical_transform[col].std(ddof=0)
sns.stripplot(data = numerical_transform, x='Gross sales_zscore', dodge=True)

In [ ]:
sns.boxplot(data=numerical_transform, x='Gross sales')

## Categorical
### Summary Table

In [ ]:
counts = raw_df['ID Salesman'].value_counts()
counts

In [ ]:
raw_df['ID Salesman'].value_counts(normalize=True)

In [ ]:
summarize = raw_df.groupby('ID Salesman')['Total Sales'].sum()
summarize.sort_values(ascending=False)

In [ ]:
# rata - rata per Salesman
mean_sales = (summarize / counts).sort_values(ascending=False)
mean_sales

In [ ]:
counts.plot.bar()

In [ ]:
summarize.plot.bar()

In [ ]:
mean_sales.plot.bar()

### Cross Tab

In [ ]:
cont_tab1 = pd.crosstab(raw_df['ID Salesman'], raw_df['Nama Pelanggan'])
cont_tab1

In [ ]:
cont_tab2 = pd.crosstab(raw_df['ID Salesman'], raw_df['Jenis Kirim'],normalize='columns')
cont_tab2

In [ ]:
labels = lambda x: x[1][:5] if cont_tab1.stack()[int(x[0])][x[1]] > 10 else "" 

mosaic(cont_tab1.stack(), labelizer=labels)
plt.show()

In [ ]:
cont_tab2.plot.bar()

### Categorical vs Numerical

In [ ]:
num_var = 'Gross sales'
cat_var = 'ID Salesman'
concat_df = pd.DataFrame([numerical_transform[num_var], raw_df[cat_var]]).T

#Filtering category yang akan di visualisasi
# mask = concat_df[cat_var].isin([669618,848311])
# concat_df = concat_df[mask]
concat_df.head()

In [ ]:
# separate histogram
sns.histplot(hue=cat_var, x=num_var, data=concat_df)
plt.show()

In [ ]:
# separate boxplot
sns.catplot(x=cat_var, y=num_var, data=concat_df,kind = 'box')
plt.show()

In [ ]:
sns.ecdfplot(data=concat_df, x=num_var, hue=cat_var, palette='plasma')
plt.show()